In [ ]:
Tokenizing - 컴퓨터가 이해할 수 있는 단위
- sent_tokenize(구두점)
- word_tokenize(구두점), regexp_tokenize, TweetTokenize
- Stemming(어간분리): stem+y,s,ed,...
- Lemmatization(어근추출): 접사+어근+접사
- Morpheme Analyze(형태소분석) + POS Tagger(품사부착)
- Entropy, Perplexity
- BPE(빈도) => WPM(확률-PMI) => SPM

Normalization - 일반화 + 불용어

오탈자 - Edit distance(편집거리), 초중종=1음절

=> 데이터분석 + 모델링

In [1]:
from nltk.stem import PorterStemmer

In [2]:
stemmer = PorterStemmer()

In [3]:
stemmer.stem('study studied')

'study studi'

In [ ]:
# BPE; 자주 나오는 표현에 짧은 코드를 부여하자 => 자주 나오는 표현(쌍)을 하나의 토큰처럼 사용하자
1. 데이터셋 => 단어:빈도, 단어 => split 대상(대한민국 => '대 한 민 국')
2. 쌍 중에 최고빈도 쌍 찾기 => 하나의 토큰으로 병합할 대상
3. 데이터셋 적용
4. 2번으로 다시(적절한 토큰수가 될때까지)
   100만 => 10만개

In [4]:
data = '''
low low low low low
lowest lowest
newer newer newer newer newer newer
wider wider wider
'''

In [5]:
def preprocessing(c):
    result = {}
    
    for t in c.split():
        t = ' '.join(t)+' </w>'
        if t not in result:
            result[t] = 0
        result[t] += 1

    return result

In [6]:
def findpair(tokens, n=2):
    result = {}
    for k,v in tokens.items():
        k = k.split()
        for i in range(len(k)-(n-1)):
            p = ' '.join(k[i:i+n])
            if p not in result:
                result[p] = 0
            result[p] += v
    return result

In [7]:
def findbest(pairs):
    return max(pairs, key=pairs.get)

In [8]:
import re

def mergepair(tokens, best):
    result = {}
    for k,v in tokens.items():
        nk = re.sub(re.escape(best), ''.join(best.split()), k)
        #            . => 아무문자, 문자 취급위해 re.escape
        result[nk] = v
    return result

In [9]:
def tokencount(tokens):
    tlist = []
    for t in tokens.keys():
        tlist.extend(re.sub(r'</w>', '', t).split())
    return list(set(tlist))

In [10]:
tokens = preprocessing(data)

for i in range(100):
    pairs = findpair(tokens)
    best = findbest(pairs)
    tokens = mergepair(tokens, best)
    
    if len(tokencount(tokens)) < 8:
        break
tokens

{'low</w>': 5, 'low e s t </w>': 2, 'newer</w>': 6, 'wid er</w>': 3}

In [11]:
from konlpy.corpus import kolaw
corpus = kolaw.open(kolaw.fileids()[0]).read()

In [12]:
tokens = preprocessing(corpus)
len(tokencount(tokens))

378

In [13]:
tokens = preprocessing(corpus)
bestpairs = list()

for i in range(100):
    pairs = findpair(tokens)
    best = findbest(pairs)
    tokens = mergepair(tokens, best)

    bestpairs.append(best)
    
    if len(tokencount(tokens)) < 370:
        break

In [14]:
len(tokencount(tokens)) #[t for t in filter(lambda t:len(t)>1, tokencount(tokens))]

444

In [15]:
from string import punctuation

In [ ]:
from nltk.tokenize import word_tokenize

s = "i'd like to learn more somthing. i like to learn more somthing. Finland's capital"

list(set(s.split())), len(list(set(s.split()))), \
list(set(re.sub('[{}]'.format(re.escape(punctuation)), '', s).split())), len(list(set(re.sub('[{}]'.format(re.escape(punctuation)), '', s).split()))), \
list(set(re.sub('[{}]'.format(re.escape(punctuation)), ' ', s).split())), len(list(set(re.sub('[{}]'.format(re.escape(punctuation)), ' ', s).split()))), \
list(set(word_tokenize(s))), len(list(set(word_tokenize(s))))

In [16]:
from nltk.corpus import stopwords
stop = stopwords.open('english').read()

In [17]:
s = 'I like you!'
s = 'To be or not to be' # 불용어 => 조심해야 하는 이유
r = []
for t in re.sub('[{}]'.format(re.escape(punctuation)), ' ', s.lower()).split():
         # lower, remove punctuation => normalizing, split => tokenizing
    if t not in stop:
        r.append(t)
s, ' '.join(r)

('To be or not to be', '')

In [ ]:
Zipf 고빈도 어휘 => i, you, to, ...

In [18]:
# 형태소 분서기 => 코드 O, 패키지 O, konlpy O
korStop = '은, 는, 이, 가, 께, 을, 를, 고, 께서, 게, 에게'
cleanWords = list()
for term in '어머님 은 자장면 이 싫다 고 하셨어'.split():
    if term not in korStop:
        cleanWords.append(term)
' '.join(cleanWords)

'어머님 자장면 싫다 하셨어'

In [19]:
list(set('어머님 은 자장면 이 싫다 고 하셨어'.split())) # => 7

['어머님', '싫다', '하셨어', '자장면', '고', '은', '이']

In [20]:
list(set('어머님 자장면 싫다 하셨어'.split())) # => 4

['하셨어', '자장면', '어머님', '싫다']

In [21]:
import jpype

In [22]:
from konlpy.tag import Komoran

In [24]:
ma = Komoran()

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [26]:
korStop = '은, 는, 이, 가, 께, 을, 를, 고, 께서, 게, 에게'
s = '어머님은 자장면이 싫다고 하셨어'
r = []
for t in ma.morphs(s):
    if t not in korStop:
        r.append(t)
r

NameError: name 'ma' is not defined

In [27]:
import re

s = '어머님은 자장면이 싫다고 하셨어'
r = []
for t in ma.pos(s):
    # if not t[1].startswith('J'):
    if not re.match('J|E', t[1]) and len(t[0]) > 1:
        r.append(t[0])
r

NameError: name 'ma' is not defined

In [ ]:
Lemmatization => 원형복원 | Morpheme Analyzer
POS => 형식, 의존
음절수 => 1음절('수' => )
Statistical => unigram(모두 독립 => 같은 '수')
LangaugeModel => sequence('수' => 문맥에 따라 다른 '수')

In [ ]:
불용어 사전 X

In [ ]:
whitelist = ['선물 포장']
'선물포장', '선.물.포.장'

In [28]:
from string import punctuation
# 사전베이스 + 정규식
dic = ['씨발', '시발'] # 끊임없이 관리 => 누가:인턴
s = '이런 씨발 시발 시.발. 시~발! ㅅ1발 시이이이이발'
r = []
for t in re.sub('[{}]'.format(re.escape(punctuation)), '', s).split():
    if t in dic:
        r.append('*'*len(t))
    else:
        r.append(t)
' '.join(r)

'이런 ** ** ** ** ㅅ1발 시이이이이발'

In [29]:
def preprocessing(c):
    result = {}
    
    for t in c.split():
        t = '<w> '+' '.join(t)+' </w>'
        if t not in result:
            result[t] = 0
        result[t] += 1

    return result

In [30]:
data = '''
씨발 씨발 씨발 씨발 씨발
시발 시발 시발 시발 시발 시.발. 시~발! ㅅ1발 시이이이이발
'''

In [31]:
tokens = preprocessing(re.sub('[{}]'.format(re.escape(punctuation)), '', data))
tokens

{'<w> 씨 발 </w>': 5,
 '<w> 시 발 </w>': 7,
 '<w> ㅅ 1 발 </w>': 1,
 '<w> 시 이 이 이 이 발 </w>': 1}

In [32]:
tokens = preprocessing(re.sub('[{}]'.format(re.escape(punctuation)), '', data))
bestList = []
for i in range(4):
    pairs = findpair(tokens)
    best = findbest(pairs)
    tokens = mergepair(tokens, best)
    bestList.append(best)
    # if len(tokencount(tokens)) < 8:
    #     break
tokens

{'<w>씨 발</w>': 5, '<w>시발</w>': 7, '<w> ㅅ 1 발</w>': 1, '<w>시 이 이 이 이 발</w>': 1}

In [33]:
bestList

['발 </w>', '<w> 시', '<w>시 발</w>', '<w> 씨']

In [34]:
s = [re.sub(r'^<w> ', '', b) for b in filter(lambda b:re.search(r'^<w>[^>]+$', b), bestList)]
# s = [b for b in map(lambda b:''.join([t for t in b.split() if t != '<w>']), 
#      filter(lambda b:re.search(r'^<w>[^>]+$', b), bestList))]
e = [re.sub(r' </w>$', '', b) for b in filter(lambda b:re.search(r'^[^<]+</w>$', b), bestList)]
dic = [re.sub(r'</?w>|[ ]', '', b) for b in filter(lambda b:re.search(r'^<w>.+?</w>$', b), bestList)]

In [35]:
p = list()
for sop in s:
    for eop in e:
        p.append(re.compile(r'^{}.*?{}$'.format(sop, eop)))
for r in dic:
    p.append(re.compile(r'.*?{}.*?'.format(r)))
p

[re.compile(r'^시.*?발$', re.UNICODE),
 re.compile(r'^씨.*?발$', re.UNICODE),
 re.compile(r'.*?시발.*?', re.UNICODE)]

In [36]:
d = re.sub(r'[{}]'.format(re.escape(punctuation)), '', data).split()
r = []
for t in d:
    for pattern in p:
        if pattern.search(t):
            r.append(pattern.sub('*'*len(t), t))
            break
' '.join(r)

'** ** ** ** ** ** ** ** ** ** ** ** ******'